In [1]:
import datetime
import pandas as pd
import os

logs = []

def display_main_menu(is_librarian):
    print("\nLibrary Management System")
    print("1. Borrow a book")
    print("2. Return a book")
    print("3. List all books")
    print("4. Reserve a book")
    print("5. Search a book")
    if is_librarian:
        print("6. Add a new book")
        print("7. Remove a book")
        print("8. Add user")
        print("9. Remove user")
        print("10. Generate reports")
        print("11. List log operations")
        print("12. Exit")
    else:
        print("6. Exit")

def get_user_choice():
    try:
        choice = int(input("Enter your choice: "))
        return choice
    except ValueError:
        print("Please enter a valid integer.")
        return None

def log_operation(operation_type, username):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    logs.append({"timestamp": timestamp, "operation": operation_type, "username": username})

def load_library_data():
    if os.path.exists("books.csv"):
        return pd.read_csv("books.csv")
    else:
        return pd.DataFrame(columns=["title", "author", "year_of_publication"])

def save_library_data(df):
    df.to_csv("books.csv", index=False)

def load_user_data():
    if os.path.exists("user.csv"):
        return pd.read_csv("user.csv")
    else:
        return pd.DataFrame(columns=["username", "password", "role"])

def save_user_data(df):
    df.to_csv("user.csv", index=False) 

def load_borrowed_books():
    if os.path.exists("borrowed_books.csv"):
        return pd.read_csv("borrowed_books.csv")
    else:
        return pd.DataFrame(columns=["title", "borrower", "borrowed_date"])

def save_borrowed_books(df):
    df.to_csv("borrowed_books.csv", index=False)

def load_reservations():
    if os.path.exists("reservations.csv"):
        return pd.read_csv("reservations.csv")
    else:
        return pd.DataFrame(columns=["title", "user", "reservation_date"])

def save_reservations(df):
    df.to_csv("reservations.csv", index=False)

In [57]:
def handle_add_book(username):
    library_books_df = load_library_data()
    title = input("Enter the title of the book: ")
    author = input("Enter the author's name: ")
    yop = int(input("Enter the year of publication: "))
    
    new_book_df = pd.DataFrame({"title": [title], "author": [author], "year_of_publication": [yop]})
    library_books_df = pd.concat([library_books_df, new_book_df], ignore_index=True)
    save_library_data(library_books_df)
    print(f"\nBook '{title}' added successfully.\n")
    log_operation("Add Book", username)

In [58]:
def handle_remove_book(username):
    library_books_df = load_library_data()
    title = input("Enter the title of the book to remove: ").strip()
    library_books_df = library_books_df[library_books_df['title'].str.lower() != title.lower()]
    save_library_data(library_books_df)
    print(f"Book '{title}' removed successfully.")
    log_operation("Remove Book", username)

In [59]:
def handle_borrow_book(username):
    library_books_df = load_library_data()
    borrowed_books_df = load_borrowed_books()
    title = input("Enter the title of the book you want to borrow: ").strip()
    
    if title.lower() in library_books_df['title'].str.lower().values and not borrowed_books_df['title'].str.lower().eq(title.lower()).any():
        borrowed_books_df = pd.concat([borrowed_books_df, pd.DataFrame({
            "title": [title], "borrower": [username], "borrowed_date": [datetime.datetime.now()]
        })])
        save_borrowed_books(borrowed_books_df)
        print(f"You have successfully borrowed '{title}'.")
        log_operation("Borrow Book", username)
    else:
        print(f"Book '{title}' is unavailable or already borrowed.")

In [60]:
def handle_return_book(username):
    borrowed_books_df = load_borrowed_books()
    title = input("Enter the title of the book to return: ").strip()
    borrowed_books_df = borrowed_books_df[borrowed_books_df['title'].str.lower() != title.lower()]
    save_borrowed_books(borrowed_books_df)
    print(f"Book '{title}' returned successfully.")
    log_operation("Return Book", username)

In [61]:
def handle_reserve_book(username):
    library_books_df = load_library_data()
    borrowed_books_df = load_borrowed_books() 
    reservations_df = load_reservations()
    title = input("Enter the title of the book you want to reserve: ").strip()

    if title.lower() in library_books_df['title'].str.lower().values:
        if borrowed_books_df['title'].str.lower().eq(title.lower()).any():
            print(f"Book '{title}' is currently borrowed and cannot be reserved.")
        elif reservations_df['title'].str.lower().eq(title.lower()).any():
            print(f"Book '{title}' is already reserved.")
        else:
            reservations_df = pd.concat([reservations_df, pd.DataFrame({
                "title": [title], "user": [username], "reservation_date": [datetime.datetime.now()]
            })], ignore_index=True)
            save_reservations(reservations_df)
            print(f"You have successfully reserved '{title}'.")
            log_operation("Reserve Book", username)
    else:
        print(f"Book '{title}' does not exist in the library.")

In [62]:
def handle_list_books():
    library_books_df = load_library_data()
    if not library_books_df.empty:
        print("\nBooks available in the library:")
        for _, book in library_books_df.iterrows():
            print(f"Title: {book['title']}, Author: {book['author']}, Year: {book['year_of_publication']}")
    else:
        print("No books available.")

In [63]:
def handle_search_book():
    library_books_df = load_library_data()

    while True:
        print("\nSearch Book Options:")
        print("1. Search by Title")
        print("2. Search by Author")
        print("3. Search by Year of Publication")
        print("4. Exit Search")

        search_choice = get_user_choice()

        if search_choice == 1:
            by_title(library_books_df)
        elif search_choice == 2:
            by_author(library_books_df)
        elif search_choice == 3:
            by_yop(library_books_df)
        elif search_choice == 4:
            break
        else:
            print("Invalid choice. Please try again.")

def by_title(library_books_df):
    search_title = input("Enter the title: ")
    results = library_books_df[library_books_df['title'].str.lower() == search_title.lower()]

    if not results.empty:
        for _, book in results.iterrows():
            print(f"\nBook found:\nTitle: {book['title']}\nAuthor: {book['author']}\nYear of Publication: {book['year_of_publication']}")
    else:
        print(f"\nNo book found with the title '{search_title}'")

def by_author(library_books_df):
    search_author = input("Enter the author's name: ")
    results = library_books_df[library_books_df['author'].str.lower() == search_author.lower()]

    if not results.empty:
        for _, book in results.iterrows():
            print(f"\nBook found:\nTitle: {book['title']}\nAuthor: {book['author']}\nYear of Publication: {book['year_of_publication']}")
    else:
        print(f"\nNo book found with the author '{search_author}'")

def by_yop(library_books_df):
    search_yop = input("Enter the year of publication: ")
    results = library_books_df[library_books_df['year_of_publication'].astype(str) == search_yop]

    if not results.empty:
        for _, book in results.iterrows():
            print(f"\nBook found:\nTitle: {book['title']}\nAuthor: {book['author']}\nYear of Publication: {book['year_of_publication']}")
    else:
        print(f"\nNo book found with the year of publication '{search_yop}'")


In [64]:
def handle_add_user(logged_in_user):
    users_df = load_user_data()
    username = input("Enter username for new user: ")
    password = input("Enter password: ")
    role = input("Enter role (user/librarian): ").strip().lower()
    
    if role in ["user", "librarian"]:
        users_df = pd.concat([users_df, pd.DataFrame({"username": [username], "password": [password], "role": [role]})], ignore_index=True)
        save_user_data(users_df) 
        print(f"User '{username}' added as '{role}'.")
        log_operation("Add User", logged_in_user)
    else:
        print("Invalid role.")

In [65]:
def handle_remove_user(logged_in_user):
    users_df = load_user_data()
    username = input("Enter username of the user to remove: ")
    users_df = users_df[users_df['username'] != username]
    save_user_data(users_df) 
    print(f"User '{username}' removed.")
    log_operation("Remove User", logged_in_user)

In [66]:
def handle_generate_reports():
    borrowed_books_df = load_borrowed_books()
    if not borrowed_books_df.empty:
        print("\nBorrowed Books Report:")
        for _, record in borrowed_books_df.iterrows():
            print(f"Title: {record['title']}, Borrower: {record['borrower']}, Date: {record['borrowed_date']}")
    else:
        print("No books have been borrowed.")

In [67]:
def handle_list_logs():
    if logs:
        print("\nLog Operations:")
        for log in logs:
            print(f"{log['timestamp']} - {log['operation']} by {log['username']}")
    else:
        print("No log operations recorded.")

In [68]:
def main():
    users_df = load_user_data()
    username = input("Enter username: ").strip()
    password = input("Enter password: ").strip()

    user_record = users_df[(users_df['username'] == username) & (users_df['password'] == password)]
    user_record = users_df[users_df['username'].str.lower() == username.lower()]
    
    if not user_record.empty:
        role = user_record.iloc[0]['role']
        is_librarian = role == 'librarian'
        print(f"\nWelcome, {username} ({role})!\n")

        while True:
            display_main_menu(is_librarian)
            choice = get_user_choice()

            if choice == 1:
                handle_borrow_book(username)
            elif choice == 2:
                handle_return_book(username)
            elif choice == 3:
                handle_list_books()
            elif choice == 4:
                handle_reserve_book(username) 
            elif choice == 5:
                handle_search_book()
            elif is_librarian:
                if choice == 6:
                    handle_add_book(username)
                elif choice == 7:
                    handle_remove_book(username)
                elif choice == 8:
                    handle_add_user(username)
                elif choice == 9:
                    handle_remove_user(username)
                elif choice == 10:
                    handle_generate_reports()
                elif choice == 11:
                    handle_list_logs()
                elif choice == 12:
                    print("Goodbye!")
                    break
                else:
                    print("Invalid choice.")
            else:
                if choice == 5:
                    print("Goodbye!")
                    break
                else:
                    print("Invalid choice.")
    else:
        print("Invalid username or password.")

In [69]:
main()


Welcome, anitha (librarian)!


Library Management System
1. Borrow a book
2. Return a book
3. List all books
4. Reserve a book
5. Search a book
6. Add a new book
7. Remove a book
8. Add user
9. Remove user
10. Generate reports
11. List log operations
12. Exit

Search Book Options:
1. Search by Title
2. Search by Author
3. Search by Year of Publication
4. Exit Search

No book found with the author 'Arundhathi'

Search Book Options:
1. Search by Title
2. Search by Author
3. Search by Year of Publication
4. Exit Search
Invalid choice. Please try again.

Search Book Options:
1. Search by Title
2. Search by Author
3. Search by Year of Publication
4. Exit Search

Library Management System
1. Borrow a book
2. Return a book
3. List all books
4. Reserve a book
5. Search a book
6. Add a new book
7. Remove a book
8. Add user
9. Remove user
10. Generate reports
11. List log operations
12. Exit
Goodbye!
